In [15]:
#!/usr/bin/env python3


# import necessary modules
import smtplib
import pandas as pd
import os
import glob
import sys

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
# set home directory so can be used on all OS
home = os.path.expanduser('~')

In [55]:
# Find most recently exported files from registry and repository
# reg_file should be report XX
# repo_file should be report XX
# link_file should be downloaded from XX as according to wiki
reg_file = max(glob.iglob(home+'/Downloads/RDRPRegistry_DATA_*'), key=os.path.getctime)
repo_file = max(glob.iglob(home+'/Downloads/RDRPRepository_DATA_*'), key=os.path.getctime)
link_file = max(glob.iglob(home+'/Downloads/RDRPRepository_Participants_*'), key=os.path.getctime)

In [39]:
print(reg_file)
print(repo_file)
print(link_file)

/Users/douglasstrodtman/Downloads/RDRPRegistry_DATA_TEST.csv
/Users/douglasstrodtman/Downloads/RDRPRepository_DATA_TEST.csv
/Users/douglasstrodtman/Downloads/RDRPRepository_Participants_TEST.csv


In [56]:
# Load data from each using record_id as index
reg_data = pd.read_csv(reg_file, index_col='record_id')
repo_data = pd.read_csv(repo_file, index_col='record_id')
link_data = pd.read_csv(link_file, skiprows=1, header=None, names=['dummy', 'na', 
    'record_id', 'd', 'e', 'f', 'survey', 'link'], index_col='record_id', dtype=object) # headers in the file are poorly written. na, d, e, f are all not used 

# Extract links for survey queues
reg_data['link'] = link_data.link

# And for survey
reg_data['survey'] = link_data.survey

In [57]:
# Check to make sure we are sending surveys
# Script is set up to take any text formatted according to rules published on wiki
if input('Would you like to send out the survey queue? y/n ') == 'y':
    form = open(home+'/git/redcap/email_temp/survey.txt')
    form = form.read()
    sys.stdout.write("Survey script loaded\n")
else:
    sys.stdout.write('Select one of the following templates:\n')
    os.system('ls '+home+'/git/redcap/email_temp')
    selection = input('Which of the above templates would you like to load? ')
    form = open(home+'/git/redcap/email_temp/'+selection)
    form = form.read()
    sys.stdout.write(selection+' loaded')

Would you like to send out the survey queue? y/n y
Survey script loaded


In [58]:
# Check to see what subject line should be used, then set
subl_sel = input('Please select a subject line for your message:\n(1) Please complete the attached surveys\n(2) Greetings from the UW Reading & Dyslexia Research Program\n(3) New study opportunity with the UW Reading & Dyslexia Research Program\n(4) Custom\nPlease enter the number of your choice: ')
if subl_sel == '1':
    subl = 'Please complete the attached surveys'
elif subl_sel == '2':
    subl = 'Greetings from the UW Reading & Dyslexia Research Program'
elif subl_sel == '3':
    subl = 'New study opportunity UW Reading & Dyslexia Research Program'
else:
    subl = input('Please enter text for email Subject line: ')

Please select a subject line for your message:
(1) Please complete the attached surveys
(2) Greetings from the UW Reading & Dyslexia Research Program
(3) New study opportunity with the UW Reading & Dyslexia Research Program
(4) Custom
Please enter the number of your choice: 1


In [59]:
subl

'Please complete the attached surveys'

In [60]:
# Set all features not to be iterated
rdrp_email = "rdrp@uw.edu"

body = """\
<html>
  <head></head>
  <body>
    <p>{}
    </p>
  </body>
</html>
"""

In [61]:
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login("rdrp@uw.edu", input('Please enter password for rdrp@uw.edu: '))

Please enter password for rdrp@uw.edu: readingrainier214


(235, b'2.7.0 Accepted')

In [62]:
# Scrape subject data from info, input into form.format in the same order as the positions
# to be filled in.
for sub in reg_data.index:
    if pd.isnull(reg_data.parent_first_name[sub]):
        ename = reg_data.first_name[sub]
        link = reg_data.link[sub]
        your = 'your'
        body = form.format(ename=ename, link=link, your=your)
        sub_email = reg_data.email[sub]
        # Create message container - the correct MIME type is multipart/alternative.
        msg = MIMEMultipart()
        msg['Subject'] = subl
        msg['From'] = rdrp_email
        msg['To'] = sub_email
        
        # Create the body of the message.
        html = body.format(form)
        
        # Record the MIME types of text/html and attach into message container.
        msg.attach(MIMEText(html, 'html'))
        
        # sendmail function takes 3 arguments: sender's address, recipient's address
        # and message to send - here it is sent as one string.
        server.sendmail(rdrp_email, sub_email, msg.as_string())
    else:
        ename = reg_data.parent_first_name[sub]
        link = reg_data.link[sub]
        your = reg_data.first_name[sub] + "'s"
        body = form.format(ename=ename, link=link, your=your)
        sub_email = reg_data.email[sub]
        # Create message container - the correct MIME type is multipart/alternative.
        msg = MIMEMultipart()
        msg['Subject'] = subl
        msg['From'] = rdrp_email
        msg['To'] = sub_email
        
        # Create the body of the message.
        html = body.format(form)
        
        # Record the MIME types of text/html and attach into message container.
        msg.attach(MIMEText(html, 'html'))
        
        # sendmail function takes 3 arguments: sender's address, recipient's address
        # and message to send - here it is sent as one string.
        server.sendmail(rdrp_email, sub_email, msg.as_string())

#close server
server.quit()

(221, b'2.0.0 closing connection i63sm1935800pgc.26 - gsmtp')

In [53]:
reg_data.first_name

record_id
1     Hugster
2         NaN
3     Hugster
4         NaN
5     Hugster
6        Test
10        NaN
Name: first_name, dtype: object

In [54]:


# Create message container - the correct MIME type is multipart/alternative.
msg = MIMEMultipart()
msg['Subject'] = subl
msg['From'] = rdrp
msg['To'] = sub_email

# Create the body of the message.
html = body.format(form)

# Record the MIME types of text/html and attach into message container.
msg.attach(MIMEText(html, 'html'))

# Send the message via local gmail server.
s = smtplib.SMTP('smtp.gmail.com', 587)
# sendmail function takes 3 arguments: sender's address, recipient's address
# and message to send - here it is sent as one string.
s.sendmail(rdrp, sub_email, msg.as_string())
s.quit()

NameError: name 'rdrp' is not defined